In [5]:
import os
import math
import torch
import random
import shutil
import pickle
import pprint
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import models, transforms
from transformers import ViTFeatureExtractor, ViTModel, ViTImageProcessor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error

In [6]:
data_frame = pd.read_csv('oasis_cross-sectional.csv')
data_frame.drop('Delay', axis=1, inplace=True)
data_frame = data_frame.dropna()

In [7]:
print(data_frame)

                ID M/F Hand  Age  Educ  eTIV   nWBV    ASF  MMSE  CDR  SES
0    OAS1_0001_MR1   F    R   74   2.0  1344  0.743  1.306  29.0  0.0  3.0
1    OAS1_0002_MR1   F    R   55   4.0  1147  0.810  1.531  29.0  0.0  1.0
2    OAS1_0003_MR1   F    R   73   4.0  1454  0.708  1.207  27.0  0.5  3.0
8    OAS1_0010_MR1   M    R   74   5.0  1636  0.689  1.073  30.0  0.0  2.0
9    OAS1_0011_MR1   F    R   52   3.0  1321  0.827  1.329  30.0  0.0  2.0
..             ...  ..  ...  ...   ...   ...    ...    ...   ...  ...  ...
411  OAS1_0453_MR1   F    R   70   1.0  1295  0.748  1.355  29.0  0.5  4.0
412  OAS1_0454_MR1   F    R   73   3.0  1536  0.730  1.142  23.0  0.5  2.0
413  OAS1_0455_MR1   F    R   61   2.0  1354  0.825  1.297  28.0  0.0  4.0
414  OAS1_0456_MR1   M    R   61   5.0  1637  0.780  1.072  30.0  0.0  2.0
415  OAS1_0457_MR1   F    R   62   3.0  1372  0.766  1.279  26.0  0.0  3.0

[216 rows x 11 columns]


In [8]:
with open('patients_codes_set.pkl', 'rb') as f:
    patients_codes_set = pickle.load(f)

patients_codes_list = list(patients_codes_set)
print("Set loaded from patients_codes_set.pkl:", patients_codes_list)

Set loaded from patients_codes_set.pkl: ['0263', '0351', '0094', '0082', '0374', '0120', '0205', '0142', '0164', '0441', '0300', '0021', '0440', '0298', '0453', '0307', '0390', '0158', '0308', '0288', '0451', '0447', '0352', '0003', '0039', '0066', '0380', '0238', '0335', '0123', '0023', '0243', '0400', '0226', '0454', '0304', '0267', '0290', '0247', '0418', '0402', '0210', '0286', '0143', '0179', '0233', '0042', '0315', '0272', '0016', '0022', '0161', '0273', '0115', '0287', '0329', '0411', '0432']


In [14]:
data_statistics_dataframe = pd.DataFrame(columns=['Type', 'Class', 'Number of patients', 'number of male', 'number of female', 'Age mean', 'Age std', 'Educ mean', 'Educ std', 'eTIV mean', 'eTIV std', 'nWBV mean', 'nWBV std', 'ASF mean', 'ASF std', 'MMSE mean', 'MMSE std', 'CDR mean', 'CDR std', 'SES mean', 'SES std'])

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']

folders = './stock'
for mode in mode_list:
    # print("Mode:", mode)
    for folder in os.listdir(folders):
        patients_list = []
        if os.path.isdir(folders + '/'+folder):
            # print("Folder:", folder)
            folder_path = folders+'/'+folder
            files = [f for f in os.listdir(folder_path) if f.endswith('.png') and mode in f]
            files_code = list(set([f[5:9] for f in files]))
            inter = patients_codes_set.intersection(files_code)
            selected_rows = data_frame[data_frame['ID'].str[5:9].isin(inter)]
            num = len(selected_rows)
            male = len(selected_rows[selected_rows['M/F'] == 'M'])
            female = len(selected_rows[selected_rows['M/F'] == 'F'])
            age_mean = selected_rows['Age'].mean()
            age_std = selected_rows['Age'].std()
            edu_mean = selected_rows['Educ'].mean()
            edu_std = selected_rows['Educ'].std()
            eTIV_mean = selected_rows['eTIV'].mean()
            eTIV_std = selected_rows['eTIV'].std()
            nWBV_mean = selected_rows['nWBV'].mean()
            nWBV_std = selected_rows['nWBV'].std()
            ASF_mean = selected_rows['ASF'].mean()
            ASF_std = selected_rows['ASF'].std()
            MMSE_mean = selected_rows['MMSE'].mean()
            MMSE_std = selected_rows['MMSE'].std()
            CDR_mean = selected_rows['CDR'].mean()
            CDR_std = selected_rows['CDR'].std()
            SES_mean = selected_rows['SES'].mean()
            SES_std = selected_rows['SES'].std()

            new_data = pd.DataFrame([{'Type': mode, 'Class': folder, 'Number of patients': num, 'number of male': male, 'number of female': female, 'Age mean': age_mean, 'Age std': age_std, 'Educ mean': edu_mean, 'Educ std': edu_std, 'eTIV mean': eTIV_mean, 'eTIV std': eTIV_std, 'nWBV mean': nWBV_mean, 'nWBV std': nWBV_std, 'ASF mean': ASF_mean, 'ASF std': ASF_std, 'MMSE mean': MMSE_mean, 'MMSE std': MMSE_std, 'CDR mean': CDR_mean, 'CDR std': CDR_std, 'SES mean': SES_mean, 'SES std': SES_std}])

            # print("Files code:", files_code)
            data_statistics_dataframe = pd.concat([data_statistics_dataframe, new_data], ignore_index=True)
data_statistics_dataframe.to_csv('data_statistics.csv', index=False)

/var/folders/py/pdfb_cjs1fxfr2jkwq245b6m0000gn/T/ipykernel_34423/2275670922.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_statistics_dataframe = pd.concat([data_statistics_dataframe, new_data], ignore_index=True)


In [15]:
data_statistics_dataframe

,Type,Class,Number of patients,number of male,number of female,Age mean,Age std,Educ mean,Educ std,eTIV mean,...,nWBV mean,nWBV std,ASF mean,ASF std,MMSE mean,MMSE std,CDR mean,CDR std,SES mean,SES std
0,MR1_1,VeryMildDemented,55,25,30,76.927273,7.143644,3.036364,1.373878,1497.272727,...,0.729473,0.036771,1.190436,0.146515,25.800000,3.285208,0.500000,0.000000,2.654545,1.142093
1,MR1_1,ModerateDemented,22,8,14,77.818182,6.730431,3.045455,1.396502,1446.727273,...,0.722909,0.038988,1.232636,0.153698,24.000000,4.690416,0.636364,0.441367,2.590909,1.098011
2,MR1_1,MildDemented,29,14,15,77.034483,7.528435,2.896552,1.318680,1528.827586,...,0.726897,0.041238,1.163793,0.134910,25.724138,3.483600,0.500000,0.000000,2.689655,1.137147
3,MR1_1,NonDemented,26,14,12,77.076923,7.924257,3.153846,1.515053,1523.846154,...,0.726769,0.038138,1.168269,0.140468,26.384615,3.007682,0.500000,0.000000,2.538462,1.303250
4,MR1_2,VeryMildDemented,56,26,30,77.053571,7.141224,3.035714,1.361340,1497.267857,...,0.728821,0.036760,1.190107,0.145198,25.857143,3.283172,0.500000,0.000000,2.660714,1.132604
5,MR1_2,ModerateDemented,23,9,14,78.086957,6.700835,3.043478,1.364427,1448.913043,...,0.721609,0.038599,1.230000,0.150696,24.217391,4.699676,0.630435,0.432156,2.608696,1.076152
6,MR1_2,MildDemented,29,14,15,77.034483,7.528435,2.896552,1.318680,1528.827586,...,0.726897,0.041238,1.163793,0.134910,25.724138,3.483600,0.500000,0.000000,2.689655,1.137147
7,MR1_2,NonDemented,26,14,12,77.076923,7.924257,3.153846,1.515053,1523.846154,...,0.726769,0.038138,1.168269,0.140468,26.384615,3.007682,0.500000,0.000000,2.538462,1.303250
8,MR1_3,VeryMildDemented,56,26,30,77.053571,7.141224,3.035714,1.361340,1497.267857,...,0.728821,0.036760,1.190107,0.145198,25.857143,3.283172,0.500000,0.000000,2.660714,1.132604
9,MR1_3,ModerateDemented,23,10,13,78.434783,6.521476,3.000000,1.314257,1465.260870,...,0.720174,0.038420,1.217609,0.155960,24.260870,4.750286,0.630435,0.432156,2.608696,1.076152
